In [ ]:
#default_exp indexer

# Indexer

Annoy treemap or FAISS or other solutions. Given a dataset of tensors, returns a dictionary or database or treemap structure, something that is searchable for later. It would be nice to be able to diff this somehow, or make sure that it's up-to-date. Maybe keeping two copies is okay? One for backup and quick-searching, one for main search once it's indexed any new images. 

This executor `needs` both Encoder and Loader to send it the new and old vectors, respectively. So it needs to be preceded by some kind of **join_all** component that can makesure we're not missing new data before handing it over to the indexer. Hm

In [ ]:
#export
def join_all(db, new_files, new_embeddings):
    for i, file in enumerate(new_files):
        path, slug = file
        start = len(db)
        index = i + start
        db[slug] = {
            'slug': slug,
            'fpath': path,
            'embed': new_embeddings[i],
            'index': index
        }
    return(db)

And build treemap

In [ ]:
#export
from annoy import AnnoyIndex

In [ ]:
#export
def build_treemap(db):
    treemap = AnnoyIndex(512, 'angular')
    for v in db.values():
        treemap.add_item(v['index'], v['embed'])

    # Build the treemap, with 5 trees rn
    treemap.build(5)

    return(treemap)
    

In [ ]:
t = build_treemap(db)

In [ ]:
t.get_n_items(), t.get_n_trees()

(3698, 5)

In [ ]:
#export
import torch

In [ ]:
#export
def save_archives(root, treemap, db):
    dbpath = root/'memery.pt'
    if dbpath.exists():
#         dbpath.rename(root/'memery-bak.pt')
        dbpath.unlink()
    torch.save(db, dbpath)
    
    treepath = root/'memery.ann'
    if treepath.exists():
#         treepath.rename(root/'memery-bak.ann')
        treepath.unlink()
    treemap.save(str(treepath))
    
    return(str(dbpath), str(treepath))

In [ ]:
save_archives(root, t, db)

('/home/mage/Pictures/memes/memery.pt', '/home/mage/Pictures/memes/memery.ann')